In [11]:
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, MobileNet, InceptionV3, EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import json



In [2]:
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32

train_dir = "../Dataset_fish_extracted/train"
val_dir = "../Dataset_fish_extracted/val"
test_dir = "../Dataset_fish_extracted/test"

#Training generator with augmentation
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

# Rescaling the val and test data
val_test_gen = ImageDataGenerator(rescale=1./255)
# Create Generators
train_generator = train_gen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_test_gen.flow_from_directory(
    val_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = val_test_gen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)


Found 6225 images belonging to 11 classes.
Found 1092 images belonging to 11 classes.
Found 3187 images belonging to 11 classes.


In [3]:
# Saving the class index as json file
class_names = sorted(os.listdir(train_dir))
class_names = [name for name in class_names if os.path.isdir(os.path.join(train_dir, name))]
labels = {i: name for i, name in enumerate(class_names)}

with open('class_labels.json', 'w') as f:
    json.dump(labels, f)

## Model Training

### CUSTOM MADE CNN MODEL

In [4]:
# CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')  # Output layer
])

c:\sanju\Fish_Image_Classification_Project\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
# Train the model
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator
)

Epoch 1/20
195/195 ━━━━━━━━━━━━━━━━━━━━ 513s 3s/step - accuracy: 0.4622 - loss: 1.5798 - val_accuracy: 0.1740 - val_loss: 7.5451
Epoch 2/20
195/195 ━━━━━━━━━━━━━━━━━━━━ 525s 3s/step - accuracy: 0.5899 - loss: 1.2114 - val_accuracy: 0.5000 - val_loss: 2.0230
Epoch 3/20
195/195 ━━━━━━━━━━━━━━━━━━━━ 530s 3s/step - accuracy: 0.6602 - loss: 1.0032 - val_accuracy: 0.8288 - val_loss: 0.6153
Epoch 4/20
195/195 ━━━━━━━━━━━━━━━━━━━━ 583s 3s/step - accuracy: 0.6969 - loss: 0.8941 - val_accuracy: 0.8892 - val_loss: 0.4091
Epoch 5/20
195/195 ━━━━━━━━━━━━━━━━━━━━ 528s 3s/step - accuracy: 0.7399 - loss: 0.7705 - val_accuracy: 0.8718 - val_loss: 0.4497
Epoch 6/20
195/195 ━━━━━━━━━━━━━━━━━━━━ 535s 3s/step - accuracy: 0.7754 - loss: 0.6698 - val_accuracy: 0.8773 - val_loss: 0.4226
Epoch 7/20
195/195 ━━━━━━━━━━━━━━━━━━━━ 514s 3s/step - accuracy: 0.7863 - loss: 0.6418 - val_accuracy: 0.8901 - val_loss: 0.3733
Epoch 8/20
195/195 ━━━━━━━━━━━━━━━━━━━━ 526s 3s/step - accuracy: 0.8173 - loss: 0.5726 - val_accu

In [9]:
#Saving the trained model 
model.save('CustomCNN/custom_cnn.h5')

In [10]:
# Saving the model as keras
model.save('CustomCNN/custom_cnn.keras')

## Working on Pre Trained CNN models

### Model 1: VGG16

In [12]:
# Load pre-trained VGG16

VGG_base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step


In [13]:
# Modifying the output layer

x = VGG_base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(train_generator.num_classes, activation='softmax')(x)

model_vgg = Model(inputs=VGG_base_model.input, outputs=output)

In [14]:
# Set the whole model as trainable
VGG_base_model.trainable = True

# Freeze all layers except the last 4
for layer in VGG_base_model.layers[:-4]:
    layer.trainable = False

# Compile the model
model_vgg.compile(optimizer=Adam(learning_rate=1e-5),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

In [15]:


# Train the fine tuned VGG16 model

history_vgg_finetune = model_vgg.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5
)



Epoch 1/5
195/195 ━━━━━━━━━━━━━━━━━━━━ 2373s 12s/step - accuracy: 0.3133 - loss: 2.0011 - val_accuracy: 0.7079 - val_loss: 1.3619
Epoch 2/5
195/195 ━━━━━━━━━━━━━━━━━━━━ 2698s 14s/step - accuracy: 0.6072 - loss: 1.1959 - val_accuracy: 0.8214 - val_loss: 0.7490
Epoch 3/5
195/195 ━━━━━━━━━━━━━━━━━━━━ 2559s 13s/step - accuracy: 0.7414 - loss: 0.8234 - val_accuracy: 0.9203 - val_loss: 0.4379
Epoch 4/5
195/195 ━━━━━━━━━━━━━━━━━━━━ 2231s 11s/step - accuracy: 0.8180 - loss: 0.5917 - val_accuracy: 0.9460 - val_loss: 0.3113
Epoch 5/5
195/195 ━━━━━━━━━━━━━━━━━━━━ 1839s 9s/step - accuracy: 0.8654 - loss: 0.4507 - val_accuracy: 0.9570 - val_loss: 0.2385


In [17]:
# Saving the model as h5
model_vgg.save('VGG16/vgg_finetuned_model.h5')



In [18]:
# Saving the model as keras
model_vgg.save('VGG16/vgg_finetuned_model.keras')

### 
Model 2: ResNet50


In [16]:
#creating the base model
ResNet50_base_model = ResNet50(weights = '')

ValueError: The `weights` argument should be either `None` (random initialization), 'imagenet' (pre-training on ImageNet), or the path to the weights file to be loaded.  Received: weights=